# Credit Risk Assessment: SHAP Analysis

---

In [2]:
from aura.utils.pathing import models, reports, root
import joblib
import shap 
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import date
from scipy import sparse
import warnings
import pickle as pk
warnings.filterwarnings("ignore")
stamp=date.today().isoformat()
data = Path("../data/processed")
models = Path("../models")
reports = Path("../reports")
figs = Path("../reports/figs")
model_version = "v1"

### Load Model and Data

In [3]:
cal = joblib.load(models/f"lgbm_calibrated_{model_version}.joblib")
lgbm = cal.calibrated_classifiers_[0].estimator
X_train = pd.read_pickle(data/"X_train.pkl")
feature_order = X_train.columns.tolist()

In [4]:
dt_cols  = X_train.select_dtypes("datetime64[ns]").columns
if len(dt_cols):
    print("Dropping datetime columns for SHAP:", list(dt_cols))
    X_train = X_train.drop(columns=dt_cols)
    feature_order = [c for c in feature_order if c not in dt_cols]

feature_names = np.array(feature_order)
print(f"SHAP will run on {X_train.shape[0]:,} rows × {X_train.shape[1]} features")

Dropping datetime columns for SHAP: ['issue_d', 'earliest_cr_line', 'next_pymnt_d']
SHAP will run on 1,132,562 rows × 183 features


### SHAP Analysis

In [5]:
shap_idx_path = models/f"shap_topidx_{model_version}.joblib"
shap_vals_path = models / f"shap_vals_train_{model_version}.npz"
if shap_idx_path.exists():
    print("Using cached SHAP artefacts")
    top_idx = joblib.load(shap_idx_path)
    shap_vals = np.load(shap_vals_path)["shap_vals"]
else:
    print("Computing SHAP values...")
    explainer = shap.TreeExplainer(lgbm)
    shap_vals = explainer.shap_values(X_train, check_additivity=False)
    if isinstance(shap_vals,list): shap_vals=shap_vals[0]

    top_idx = np.argsort(np.abs(shap_vals).mean(0))[::-1][:50]
    joblib.dump(top_idx, shap_idx_path)
    np.savez_compressed(shap_vals_path, shap_vals=shap_vals.astype("float32"))

    shap.summary_plot(shap_vals, X_train, feature_names=feature_names,
                      max_display=50, show=False)
    plt.savefig(reports/f"figs/shap_summary_lightgbm_{model_version}.png", dpi=300, bbox_inches="tight")
    pd.Series(np.abs(shap_vals).mean(0)[top_idx], index=np.array(feature_names)[top_idx]) \
        .to_csv(reports / f"top_drivers_{model_version}.csv")

    print("SHAP artefacts saved")

Using cached SHAP artefacts


### Extract and Save Decorrelated Top Features

In [17]:
def encode(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.select_dtypes(["object", "category"]).columns:
        out[c] = out[c].astype("category").cat.codes.astype("int32")
    return out

def greedy(cols, threshold):
    chosen = []
    for c in cols:
        if len(chosen) == k_internal:
            break
        if (not chosen) or (X_corr[chosen].corrwith(X_corr[c]).abs().max() < threshold):
            chosen.append(c)
    return chosen

ranked_cols = feature_names[top_idx].tolist()       
X_corr = encode(X_train[ranked_cols])   
k_ui = 5          
k_internal = 10          
imp_ser = pd.Series(np.abs(shap_vals).mean(0), index=feature_names)

threshold = 0.90                                     
while threshold >= 0.50:                             
    subset = greedy(ranked_cols, threshold)
    if len(subset) >= k_ui:
        break                                
    threshold -= 0.05                                

ui_inputs = subset[:k_ui]                  
model_feats = subset
importances = imp_ser.loc[ui_inputs].values                       

print(f"threshold chosen = {threshold:.2f}")
print("UI inputs :", ui_inputs)
print("Model uses:", model_feats)
for name, imp in zip(model_feats, importances):
    print(f"{name:30} {imp:.3f}")

pd.Series(model_feats).to_csv(reports/"shap_ui_subset_v1.csv", index=False)

threshold chosen = 0.90
UI inputs : ['grade_term', 'acc_open_past_24mths', 'dti_inv', 'fico_mid_sq', 'avg_cur_bal']
Model uses: ['grade_term', 'acc_open_past_24mths', 'dti_inv', 'fico_mid_sq', 'avg_cur_bal', 'annual_inc', 'next_pymnt_d_missing', 'all_util_missing', 'home_ownership', 'mort_acc']
grade_term                     0.348
acc_open_past_24mths           0.063
dti_inv                        0.056
fico_mid_sq                    0.047
avg_cur_bal                    0.045


### All Features and their Importances

In [8]:
all_importances = np.abs(shap_vals).mean(0)
sorted_idx = np.argsort(-all_importances) 
sorted_names = np.array(feature_names)[sorted_idx]
sorted_importances = all_importances[sorted_idx]

for name, imp in zip(sorted_names, sorted_importances):
    print(f"{name}: {imp:.4f}")

grade_term: 0.3485
sub_grade: 0.1580
acc_open_past_24mths: 0.0627
dti_inv: 0.0561
fico_mid_sq: 0.0471
avg_cur_bal: 0.0445
annual_inc: 0.0347
next_pymnt_d_missing: 0.0327
all_util_missing: 0.0313
home_ownership: 0.0288
mort_acc: 0.0280
term: 0.0262
emp_length_missing: 0.0184
inq_last_12m_missing: 0.0175
inq_fi_missing: 0.0157
int_minus_subgrade_mean: 0.0132
num_actv_rev_tl: 0.0110
open_rv_24m: 0.0107
grade: 0.0106
loan_amnt: 0.0088
installment: 0.0084
tot_hi_cred_lim: 0.0083
total_bc_limit: 0.0062
deferral_term_missing: 0.0060
funded_amnt: 0.0047
mo_sin_old_rev_tl_op: 0.0046
num_rev_tl_bal_gt_0: 0.0040
purpose_emp_len: 0.0039
mths_since_recent_bc: 0.0033
avg_cur_bal_missing: 0.0029
open_acc_6m_missing: 0.0022
max_bal_bc: 0.0021
payment_plan_start_date_missing: 0.0019
log_loan_sq: 0.0018
mths_since_recent_inq: 0.0015
mo_sin_old_rev_tl_op_missing: 0.0014
total_cu_tl_missing: 0.0011
verification_status: 0.0010
num_tl_op_past_12m: 0.0009
inq_last_6mths: 0.0009
total_bal_il: 0.0007
total_il_